<a href="https://colab.research.google.com/github/Rami-RK/HugingFace_Transformers/blob/main/F_Tuning_T5_Dialogue_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine Tune a Seq2Seq (T5) Model for Summerization**

### **Objectives**
* Fine tune ["facebook/bart-large-cnn"](https://huggingface.co/facebook/bart-large-cnn) (T5) on the ["samsum"](https://huggingface.co/datasets/samsum) dataset for summerization

* Use finetuned model for inference

Summarization creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task.

#### **Installation**

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install accelerate -U # or pip install transformers[torch] --> both transformer and accelerate in one go


#### **Load Model & Tokenizer**

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

"""
Look into Model card - 400 Million parameters
BART HAS 400M PARAMS: https://github.com/facebookresearch/fairseq/tree/main/examples/bart
"""
checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

### **Load Dataset**

In [4]:
!pip install py7zr #need to install for samsum dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.6 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

dataset = load_dataset("samsum")
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

#### **Testing the pre-trained model**

In [6]:
sample = dataset['test'][0]['dialogue']
label = dataset['test'][0]['summary']

def generate_summary(input, llm):
  input_prompt = f"""
                  Summarize the following conversation.

                  {input}

                  Summary:
                  """

  input_ids = tokenizer(input_prompt, return_tensors='pt')
  tokenized_output = llm.generate(input_ids['input_ids'], min_length=30, max_length=200)
  output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)

  return output

output = generate_summary(sample, llm=model)
print("Sample")
print(sample)
print("-------------------")
print("Model Generated Summary:")
print(output)
print("Correct Summary:")
print(label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------
Model Generated Summary:
Hannah asks Amanda for Betty's number. Amanda tries to find the number but can't find it. She asks Hannah to text Larry, who is a friend of Betty's. Hannah says she'd rather text him.
Correct Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


#### **Prepare Our Dataset**

In [7]:
def tokenize_inputs(example):
  start_prompt = "Summarize the following conversation.\n\n"
  end_prompt = "\n\nSummary: "
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt').input_ids

  return example

In [8]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_inputs, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'dialogue', 'summary'])
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True) # Shortening of data, could use: shuffle and select

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
print(tokenized_datasets['train'].shape)
print(tokenized_datasets['validation'].shape)
print(tokenized_datasets['test'].shape)

(148, 2)
(9, 2)
(9, 2)


In [10]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

#### **Defining Training Arguments and Trainer object**

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bart-cnn-samsum-finetuned",  # local directory
    hub_model_id="Ramendra/dialogue_Summary",  # identifier on the Hub for directly pushing to HFhub model
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    auto_find_batch_size=True,
    evaluation_strategy='epoch',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [16]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.082500,0.133822


TrainOutput(global_step=148, training_loss=0.13038595302684888, metrics={'train_runtime': 177.1671, 'train_samples_per_second': 0.835, 'train_steps_per_second': 0.835, 'total_flos': 325065690316800.0, 'train_loss': 0.13038595302684888, 'epoch': 1.0})

#### **Save the model Locally**

In [ ]:
import os
ver = 1
output_directory="./bart-cnn-samsum-finetuned"
time_now = time.time()
model_path = os.path.join(output_directory,f"tuned_model_{ver}" )
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('./bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/tokenizer_config.json',
 './bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/special_tokens_map.json',
 './bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/vocab.json',
 './bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/merges.txt',
 './bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/added_tokens.json',
 './bart-cnn-samsum-finetuned/tuned_model_1697080968.4021988/tokenizer.json')

### **Load the model from Local**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path) #
model4local = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### **Test Our Model from Local**

In [ ]:
output = generate_summary(sample, llm=model4local)

print("Sample")
print(sample)
print("-------------------")
print("Summary:")
print(output)
print("Ground Truth Summary:")
print(label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------
Summary:
Hannah asks Amanda for Betty's number. Amanda can't find it. Hannah suggests she ask Larry to call her. Amanda asks Hannah to text Larry. Hannah agrees.
Ground Truth Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


#### **Push our model to the hub**

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

'https://huggingface.co/Ramendra/dialogue_Summary/tree/main/'

#### **Testing the Model downloadd from Hub**

In [ ]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("Ramendra/dialogue_Summary")

output = generate_summary(sample, llm=loaded_model)

print("Sample")
print(sample)
print("-------------------")
print("Summary:")
print(output)
print("Ground Truth Summary:")
print(label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------
Summary:
Hannah asks Amanda for Betty's number. Amanda doesn't have it. Hannah suggests she ask Larry to call Betty. Amanda asks Hannah to text Larry, who is very nice.
Ground Truth Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


##**[Reference](https://huggingface.co/docs/transformers/tasks/summarization)**